In [1]:
cmd = '/usr/local/bin/vw'

In [15]:
# coding: utf-8
"""
train lightgbm model
"""
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import gc
import pickle
from datetime import datetime
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler, Imputer

root = '/kaggle/competitions/avito-demand-prediction/'

def dump_matrix(matrix, name):
    pickle.dump(matrix, open(root + 'features/{}.pkl'.format(name), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
def load_matrix(name):
    return pickle.load(open(root + 'features/{}.pkl'.format(name), 'rb'))

In [4]:
tr_index = load_matrix('tr_index')
te_index = load_matrix('te_index')

In [24]:
daset_num_cols = load_matrix('daset_num_cols')

In [26]:
daset_num_cols.head()

,price,image_top_1,item_seq_number,n_missing_features,avg_days_up_user,avg_times_up_user,n_user_items,region_factor,parent_category_name_factor,user_type_factor,...,r_desc_wds,r_desc_dig,r_desc_cap,r_desc_spa,r_desc_pun,r_desc_row,r_desc_emo,r_titl_des,n_char_params,n_text_wds
item_id,,,,,,,,,,,,,,,,,,,,,
3b896605f03c,6.217606,1008,12,0,6.5,1.5,5.0,99914,914200,1433965,...,0.142857,0.000000,0.000000,0.107143,0.035714,0.000000,0.0,0.607143,48,10
c81c34d3aac5,4.616121,2909,5,1,15.0,2.0,2.0,58619,231290,1433965,...,0.133333,0.000000,0.000000,0.066667,0.066667,0.000000,0.0,1.266667,36,7
a8ab6225dd06,7.314887,527,32,1,NaN,NaN,NaN,85264,914200,1433965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39,9
20bf5eea7fb5,5.861786,930,17,3,5.0,1.0,3.0,39932,72446,1433965,...,0.142857,0.053571,0.000000,0.151786,0.026786,0.017857,0.0,0.214286,32,16
e1cf6b9d5789,7.314887,657,7,1,15.0,2.0,6.0,43712,914200,1433965,...,0.166667,0.100000,0.066667,0.150000,0.050000,0.000000,0.0,0.400000,32,15


In [27]:
daset_num_cols.replace(np.inf, np.NAN, inplace=True)

In [38]:
tr_c2v_matrix = load_matrix('tr_c2v_matrix')
te_c2v_matrix = load_matrix('te_c2v_matrix')

In [39]:
tr_list = [daset_num_cols.loc[tr_index].values, tr_c2v_matrix]
X = hstack(tr_list).tocsr()

In [37]:
imputer = Imputer(strategy='median')
scaler = StandardScaler(with_mean=False)

In [40]:
imputer.fit(X)
X = imputer.transform(X)
scaler.fit(X)
X = scaler.transform(X)

In [41]:
dump_matrix(X, 'NN_X')
del tr_list, tr_c2v_matrix
gc.collect()

103

In [42]:
te_list = [daset_num_cols.loc[te_index].values, te_c2v_matrix]
X_te = hstack(te_list).tocsr()

In [43]:
X_te = imputer.transform(X_te)
X_te = scaler.transform(X_te)

In [44]:
dump_matrix(X_te, 'NN_X_te')
del te_list, te_c2v_matrix
gc.collect()

7

In [45]:
daset_num_cols.filter(regex='_cluster').head()

,image_top_1_cluster,param_1_cluster,category_name_cluster,city_cluster,day_of_week_cluster
item_id,,,,,
3b896605f03c,1,1,1,1,1
c81c34d3aac5,1,1,1,1,1
a8ab6225dd06,1,1,1,1,1
20bf5eea7fb5,1,1,1,1,1
e1cf6b9d5789,1,1,1,1,1


In [50]:
daset_num_cols['day_of_week_cluster'].unique()

array([1])